In [1]:
import torch 
import math
from torch import nn
import torch.nn.functional as F

In [2]:
def scaled_dot_product(q, k, v, mask=None):
    # q: ma trận Query 
    # k: ma trận Key 
    # v: ma trận Value 
    # mask: mặt nạ dùng để loại bỏ những phần không cần chú ý

    d_k = q.size()[-1]  
    # Lấy kích thước cuối cùng của vector q 
    
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)  
    # Nhân ma trận q với k^T để đo mức độ tương đồng giữa các query và key.
    # Chia cho căn d_k để tránh giá trị quá lớn, giúp softmax ổn định hơn.
    
    print(f"scaled.size() : {scaled.size()}")
    
    if mask is not None:
        print(f"-- ADDING MASK of shape {mask.size()} --") 
        # Nếu có mặt nạ (mask), thêm nó vào để loại bỏ những phần không cần chú ý
        scaled += mask
    
    softmaxValue = F.softmax(scaled, dim=-1)  
    # Chuẩn hóa thành phân phối xác suất bằng softmax
    
    attention = torch.matmul(softmaxValue, v)  
    # Nhân với V để lấy kết quả cuối
    
    return attention, softmaxValue 


class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model  
        # Kích thước vector đầu vào (embedding size), ví dụ: 512
        
        self.num_heads = num_heads  
        # Số head attention (multi-head), ví dụ: 8

        self.head_dim = d_model // num_heads  
        # Mỗi đầu attention xử lý một phần của d_model
        # Ví dụ: d_model = 512, num_heads = 8 => head_dim = 64

        self.qkv_layer = nn.Linear(d_model, 3 * d_model)  
        # Một lớp Linear duy nhất để sinh ra Query (Q), Key (K), và Value (V) cùng lúc
        # Đầu ra sẽ có shape: [batch_size, seq_len, 3 * d_model]

        self.linear_layer = nn.Linear(d_model, d_model)  
        # Lớp Linear cuối để gộp lại các head thành một output duy nhất


    
    def forward(self, x, mask=None):
        batch_size, max_sequence_length, d_model = x.size()  
        # Lấy shape của đầu vào x: (batch_size, seq_len, d_model)
        print(f"x.size(): {x.size()}")


        qkv = self.qkv_layer(x)
        # Dùng 1 lớp Linear để biến đổi x thành Q, K, V.
        # Shape output: (batch_size, seq_len, 3 * d_model)
        print(f"qkv.size(): {qkv.size()}")


        qkv = qkv.reshape(batch_size, max_sequence_length, self.num_heads, 3 * self.head_dim)
        # Chia thành các head riêng biệt
        # Shape mới: (batch_size, seq_len, num_heads, 3 * head_dim)
        # Mỗi token giờ có 3 vector (Q, K, V) ứng với mỗi head
        print(f"qkv.size(): {qkv.size()}")


        qkv = qkv.permute(0, 2, 1, 3)
        # Đổi trật tự tensor để đưa num_heads lên trước (để dễ xử lý attention từng head riêng biệt)
        # Shape mới: (batch_size, num_heads, seq_len, 3 * head_dim)
        print(f"qkv.size(): {qkv.size()}")


        q, k, v = qkv.chunk(3, dim=-1)
        # Chia tensor thành 3 phần theo chiều cuối cùng (Q, K, V)
        # Mỗi phần có shape: (batch_size, num_heads, seq_len, head_dim)
        print(f"q size: {q.size()}, k size: {k.size()}, v size: {v.size()}, ")


        attention, softmaxValue = scaled_dot_product(q, k, v, mask)
        # Gọi hàm attention chuẩn hóa: attention = Softmax(QK^T / sqrt(d_k)) * V
        # Trả về:
        #   - attention: giá trị đầu ra của mỗi head
        #   - softmaxValue: ma trận attention weights
        print(f"softmaxValue.size(): {softmaxValue.size()}, attention.size:{ attention.size()} ")


        attention = attention.reshape(batch_size, max_sequence_length, self.num_heads * self.head_dim)
        # Gộp lại các head về cùng 1 chiều (trả lại shape như đầu vào ban đầu)
        print(f"attention.size(): {attention.size()}")


        out = self.linear_layer(attention)
        # Truyền qua một lớp tuyến tính để tổng hợp thông tin từ các head
        print(f"out.size(): {out.size()}")
        return out




class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()

        self.parameters_shape = parameters_shape  
        # Hình dạng của vector đầu vào để áp dụng chuẩn hóa ([d_model])
        
        self.eps = eps  
        # Một giá trị rất nhỏ để tránh chia cho 0 trong khi chuẩn hóa

        self.gamma = nn.Parameter(torch.ones(parameters_shape))  
        # Tham số học được (trainable) để scale (nhân) kết quả sau khi chuẩn hóa. Ban đầu khởi tạo là 1.

        self.beta = nn.Parameter(torch.zeros(parameters_shape))  
        # Tham số học được để shift (cộng) kết quả sau khi chuẩn hóa. Ban đầu khởi tạo là 0.



    def forward(self, inputs):
        # inputs: tensor đầu vào cần chuẩn hóa (thường có shape như [batch_size, seq_len, d_model])

        dims = [-(i + 1) for i in range(len(self.parameters_shape))]  
        # Tạo danh sách chiều để tính trung bình và phương sai.
        # Ví dụ: nếu shape là [512], thì dims = [-1], tức là chuẩn hóa theo chiều cuối cùng (d_model)

        mean = inputs.mean(dim=dims, keepdim=True)  
        # Tính trung bình của các giá trị theo chiều được chỉ định
        print(f"Mean ({mean.size()})")


        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)  
        # Tính phương sai: trung bình của bình phương sai lệch
        
        std = (var + self.eps).sqrt()  
        # Lấy căn bậc hai của phương sai để ra độ lệch chuẩn (standard deviation)
        print(f"Standard Deviation  ({std.size()})")


        y = (inputs - mean) / std  
        # Chuẩn hóa dữ liệu: trừ trung bình rồi chia cho độ lệch chuẩn
        print(f"y: {y.size()})")


        out = self.gamma * y + self.beta  
        # Scale và shift đầu ra bằng gamma và beta
        print(f"self.gamma: {self.gamma.size()}, self.beta: {self.beta.size()}")
        print(f"out: {out.size()}")
        return out  # Trả về kết quả chuẩn hóa


  
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        # d_model: số chiều của vector đặc trưng đầu vào (512).
        # hidden: số chiều của lớp ẩn (2048).
        # drop_prob: xác suất dropout để tránh overfitting (0.1).
        
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)  
        # Lớp tuyến tính đầu tiên: mở rộng chiều từ d_model -> hidden (512 -> 2048)
        
        self.linear2 = nn.Linear(hidden, d_model)  
        # Lớp tuyến tính thứ hai: thu nhỏ chiều từ hidden -> d_model (2048 -> 512)

        self.relu = nn.ReLU()  
        # Hàm kích hoạt phi tuyến ReLU

        self.dropout = nn.Dropout(p=drop_prob)  
        # Dropout để giảm overfitting bằng cách "tắt" ngẫu nhiên một số neurons trong quá trình training


    def forward(self, x):
        x = self.linear1(x)  
        # Đầu vào x có shape: (batch_size, seq_len, d_model)
        # Sau lớp linear1: shape -> (batch_size, seq_len, hidden)
        print(f"x after first linear layer: {x.size()}")


        x = self.relu(x)  
        # Áp dụng hàm ReLU để tăng tính phi tuyến
        print(f"x after activation: {x.size()}")


        x = self.dropout(x)  
        # Dropout để tránh overfitting
        print(f"x after dropout: {x.size()}")


        x = self.linear2(x)  
        # Đưa dữ liệu về lại shape ban đầu: (batch_size, seq_len, d_model)
        print(f"x after 2nd linear layer: {x.size()}")
        
        return x  # Trả về kết quả cuối cùng sau khi đi qua khối FFN




class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        # d_model: số chiều của vector đặc trưng đầu vào (512).
        # ffn_hidden: số chiều lớp ẩn trong Feed Forward Network (2048).
        # num_heads: số head trong multi-head attention (8).
        # drop_prob: xác suất dropout để giảm overfitting.
        
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        # Lớp multi-head attention – mô hình học được mối quan hệ giữa các từ trong chuỗi

        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        # Chuẩn hóa đầu ra sau attention + residual connection

        self.dropout1 = nn.Dropout(p=drop_prob)
        # Dropout giúp mô hình không bị overfit


        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        # Mạng FFN gồm 2 lớp tuyến tính áp dụng cho từng vị trí riêng biệt

        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        # Chuẩn hóa sau khối FFN

        self.dropout2 = nn.Dropout(p=drop_prob)
        # Dropout sau FFN



    def forward(self, x):
        residual_x = x
        # Lưu lại đầu vào gốc để sử dụng cho residual connection (kết nối tắt)

        print("------- ATTENTION 1 ------")
        x = self.attention(x, mask=None)
        # Áp dụng multi-head attention cho đầu vào

        print("------- DROPOUT 1 ------")
        x = self.dropout1(x)
        # Áp dụng dropout sau attention

        print("------- ADD AND LAYER NORMALIZATION 1 ------")
        x = self.norm1(x + residual_x)
        # Cộng đầu vào gốc (residual) với kết quả attention, rồi chuẩn hóa layer


        residual_x = x
        # Cập nhật residual mới là đầu ra sau khối attention

        print("------- FFN ------")
        x = self.ffn(x)
        # Truyền đầu vào qua mạng Feed Forward Network

        print("------- DROPOUT 2 ------")
        x = self.dropout2(x)
        # Dropout sau FFN

        print("------- ADD AND LAYER NORMALIZATION 2 ------")
        x = self.norm2(x + residual_x)
        # Cộng residual sau FFN và chuẩn hóa
        
        return x
        # Trả về đầu ra của lớp encoder layer sau attention và FFN



class Encoder(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob, num_layers):
        # d_model: số chiều của vector đặc trưng đầu vào (512).
        # ffn_hidden: số chiều lớp ẩn trong Feed Forward Network (2048).
        # num_heads: số đầu trong Multi-Head Attention (8).
        # drop_prob: xác suất dropout (0.1).
        # num_layers: số lớp EncoderLayer được xếp chồng (5).
        super().__init__()
        self.layers = nn.Sequential(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                     for _ in range(num_layers)])
        # Dùng list comprehension để tạo danh sách num_layers lớp EncoderLayer.
        # nn.Sequential giúp tự động truyền dữ liệu qua từng lớp encoder tuần tự.
        # Ví dụ: nếu num_layers = 5, thì self.layers sẽ chứa 5 lớp encoder liên tiếp.

    def forward(self, x):
        x = self.layers(x)  # Truyền dữ liệu đầu vào x qua toàn bộ các lớp encoder
        return x            # Trả về kết quả sau khi mã hóa


In [3]:
d_model = 512               # Kích thước vector đặc trưng (embedding) cho mỗi token.
num_heads = 8               # Số head attention. Mỗi head học một khía cạnh khác nhau của ngữ cảnh.
drop_prob = 0.1             # Xác suất dropout để giảm overfitting trong quá trình huấn luyện.
batch_size = 30             # Kích thước mỗi batch dữ liệu (số lượng câu trong một lần huấn luyện).
max_sequence_length = 200   # Độ dài tối đa của chuỗi đầu vào (số token tối đa cho mỗi câu).
ffn_hidden = 2048           # Số chiều của lớp ẩn trong Feed Forward Network. 
num_layers = 5              # Số lớp EncoderLayer được xếp chồng lên nhau.

encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers)
# Khởi tạo mô hình Encoder với các tham số trên. 
# Mô hình này sẽ có 5 lớp encoder, mỗi lớp gồm Multi-head attention, LayerNorm, FeedForward và dropout.


In [4]:
x = torch.randn((batch_size, max_sequence_length, d_model))  
# Tạo tensor đầu vào ngẫu nhiên có shape (30, 200, 512)
# Mỗi chuỗi (sequence) có độ dài 200 token, mỗi token được biểu diễn bởi vector 512 chiều.
# Dữ liệu này mô phỏng embedding + positional encoding đã được áp dụng.

out = encoder(x)  
# Truyền đầu vào x qua encoder. 
# Dữ liệu sẽ đi qua 5 lớp EncoderLayer gồm attention, chuẩn hóa, FFN và dropout.
# Kết quả là tensor có cùng shape (30, 200, 512) nhưng đã được mã hóa ngữ cảnh.


------- ATTENTION 1 ------
x.size(): torch.Size([30, 200, 512])
qkv.size(): torch.Size([30, 200, 1536])
qkv.size(): torch.Size([30, 200, 8, 192])
qkv.size(): torch.Size([30, 8, 200, 192])
q size: torch.Size([30, 8, 200, 64]), k size: torch.Size([30, 8, 200, 64]), v size: torch.Size([30, 8, 200, 64]), 
scaled.size() : torch.Size([30, 8, 200, 200])
softmaxValue.size(): torch.Size([30, 8, 200, 200]), attention.size:torch.Size([30, 8, 200, 64]) 
attention.size(): torch.Size([30, 200, 512])
out.size(): torch.Size([30, 200, 512])
------- DROPOUT 1 ------
------- ADD AND LAYER NORMALIZATION 1 ------
Mean (torch.Size([30, 200, 1]))
Standard Deviation  (torch.Size([30, 200, 1]))
y: torch.Size([30, 200, 512]))
self.gamma: torch.Size([512]), self.beta: torch.Size([512])
out: torch.Size([30, 200, 512])
------- FFN ------
x after first linear layer: torch.Size([30, 200, 2048])
x after activation: torch.Size([30, 200, 2048])
x after dropout: torch.Size([30, 200, 2048])
x after 2nd linear layer: torc